In [108]:
from re import X

import numpy as np
import pandas as pd

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [109]:
df_ndvi = pd.read_csv("data/train/NDVI.csv", encoding="windows-1251").drop(columns=["index"])
df_nir = pd.read_csv("data/train/B8A.csv", encoding="windows-1251").drop(columns=["index", "culture"]).add_suffix("_nir")
df_swir = pd.read_csv("data/train/B12.csv", encoding="windows-1251").drop(columns=["index", "culture"]).add_suffix("_swir")
df_red = pd.read_csv("data/train/B04.csv", encoding="windows-1251").drop(columns=["index", "culture"]).add_suffix("_red")
df_VegRedEdge = pd.read_csv("data/train/B05.csv", encoding="windows-1251").drop(columns=["index", "culture"]).add_suffix("_vegRedEdge")
df_blue = pd.read_csv("data/train/B02.csv", encoding="windows-1251").drop(columns=["index", "culture"]).add_suffix("_blue")
df_green = pd.read_csv("data/train/B03.csv", encoding="windows-1251").drop(columns=["index", "culture"]).add_suffix("_green")

labels = df_ndvi["culture"]
df_ndvi.drop(columns=["culture"], inplace=True)

data = pd.concat([df_ndvi, df_nir, df_swir, df_red, df_VegRedEdge, df_blue, df_green], axis=1)
data["121"].isna().sum()

610

In [110]:
class DataImputer(BaseEstimator, TransformerMixin):
    def __init__(self, impute_type: str = "mean", window_size: int = 5):
        self.impute_type = impute_type
        self.window_size = window_size + 1

    def fit(self, X: pd.DataFrame, y=None):
        return self

    def transform(self, X: pd.DataFrame, y=None) -> pd.DataFrame:
        data: pd.DataFrame = X.copy()
        if self.impute_type == "mean":
            for column in data.columns:
                data[column] = data[column].fillna(data[column].mean())
            return data

        if self.impute_type == "rolling_mean":
            for column in data.columns:
                data[column] = data[column].fillna(data[column].rolling(window=self.window_size, min_periods=1).mean())
            return data

        if self.impute_type == "ffil":
            return data.ffill()


In [111]:
from numpy import ndarray


class VegetationIndexAdder(BaseEstimator, TransformerMixin):
    def __init__(self, ndwi: bool = True, arvi: bool = True, sawi: bool = True, gemi: bool = True, ndre: bool = True, gndwi: bool = True, evi: bool = True, msavi: bool = True) -> None:
        super().__init__()

        self.N_DAYS = 26
        self.DAYS = ['121', '128', '135', '142', '149', '156', '163', '170', '177', '184', '191', '198', '205', 
                                '212', '219', '226', '233', '240', '247', '254', '261', '268', '275', '282', '289', '296']
        
        self.NDVI_START = 0
        self.NIR_START = 26
        self.SWIR_START = 52
        self.RED_START = 78
        self.VEG_REG_EDGE_START = 104
        self.BLUE_START = 130
        self.GREEN_START = 156

        self.ndwi = ndwi
        self.arvi = arvi
        self.sawi = sawi
        self.gemi = gemi
        self.ndre = ndre
        self.gndwi = gndwi
        self.evi = evi
        self.msavi = msavi



    def fit(self, X, y=None):
        return self
    
    def transform(self, X: pd.DataFrame, y=None):
        data = X.copy()

        for day in self.DAYS:
            nir = X[f"{day}_nir"]
            red = X[f"{day}_red"]
            blue = X[f"{day}_blue"]
            swir = X[f"{day}_swir"]
            veg_red_edge = X[f"{day}_vegRedEdge"]
            green = X[f"{day}_green"]

            discriminant = (2 * nir + 1) ** 2 - 8 * (nir * red)
            sqrt_value = np.sqrt(np.maximum(discriminant, 0))
            L = 1 - (2 * nir + 1 - sqrt_value) / 2

            E = (2 * (nir ** 2 - red ** 2) + 1.5 * nir + 0.5 * red) / (nir + red + 0.5)
            Rb = red - (red - blue)
            msavi_expression = (2 * nir + 1) ** 2 - 8 * (nir - red)

            data[f"{day}_ndwi"] = (nir - swir) / (nir + swir)
            data[f"{day}_arvi"] = (nir - Rb) / (nir + Rb)
            data[f"{day}_sawi"] = (nir - red) / (nir + red - L) * (1 + L)
            data[f"{day}_gemi"] = E * (1 - 0.25 * E) - ((red - 0.125) / (1 - red))
            data[f"{day}_ndre"] = (nir - veg_red_edge) / (nir + veg_red_edge)
            data[f"{day}_gndwi"] = (nir - green) / (nir + green)
            data[f"{day}_evi"] = 2.5 * (nir - red) / (nir + 6 * red - 7.5 * blue + 1)
            data[f"{day}_msavi"] = (2 * nir + 1 - np.sqrt(np.abs(msavi_expression)))


            data = data.copy()
            data.drop(columns=[f"{day}_nir", f"{day}_red", f"{day}_blue", f"{day}_swir", f"{day}_vegRedEdge", f"{day}_green"], inplace=True)


        return X.copy()


In [119]:
pipe = make_pipeline(
    DataImputer(impute_type="mean"),
    VegetationIndexAdder(),
    HistGradientBoostingClassifier()
)

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.3, random_state=0)

pipe.fit(x_train, y_train)

Pipeline(steps=[('dataimputer', DataImputer(window_size=6)),
                ('vegetationindexadder', VegetationIndexAdder()),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())])

In [120]:
predictions = pipe.predict(x_test)
print(classification_report(y_test, predictions, digits=5))

                   precision    recall  f1-score   support

           залежь    0.95455   0.99324   0.97351       296
         зерновые    0.99733   0.97650   0.98681       383
         кукуруза    1.00000   0.99718   0.99859       355
многолетние травы    0.98866   0.98866   0.98866       441
            овощи    0.99603   0.99603   0.99603       252
              соя    0.99332   0.98673   0.99001       452

         accuracy                        0.98899      2179
        macro avg    0.98832   0.98972   0.98894      2179
     weighted avg    0.98922   0.98899   0.98903      2179



In [121]:
import pickle

info = {
    "model": pipe,
    "column_suffixes": {
        "ndvi": "",
        "B8A": "_nir",
        "B12": "_swir",
        "B04": "_red",
        "B05": "_vegRedEdge",
        "B02": "_blue",
        "B03": "_green",
    },
    "f1_score": 0.989
}

with open("model.pkl", "wb") as f:
    pickle.dump(info, f)

In [123]:
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

print(model)
unpickled_predictions = model["model"].predict(x_test)
print(classification_report(y_test, unpickled_predictions, digits=5))

{'model': Pipeline(steps=[('dataimputer', DataImputer(window_size=6)),
                ('vegetationindexadder', VegetationIndexAdder()),
                ('histgradientboostingclassifier',
                 HistGradientBoostingClassifier())]), 'column_suffixes': {'ndvi': '', 'B8A': '_nir', 'B12': '_swir', 'B04': '_red', 'B05': '_vegRedEdge', 'B02': '_blue', 'B03': '_green'}, 'f1_score': 0.989}
                   precision    recall  f1-score   support

           залежь    0.95455   0.99324   0.97351       296
         зерновые    0.99733   0.97650   0.98681       383
         кукуруза    1.00000   0.99718   0.99859       355
многолетние травы    0.98866   0.98866   0.98866       441
            овощи    0.99603   0.99603   0.99603       252
              соя    0.99332   0.98673   0.99001       452

         accuracy                        0.98899      2179
        macro avg    0.98832   0.98972   0.98894      2179
     weighted avg    0.98922   0.98899   0.98903      2179

